<a href="https://colab.research.google.com/github/danielyoon-maker/ASL/blob/master/asl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-11-21 16:54:23          16110  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           9724  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           4457  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           3555  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [ ]:
! kaggle datasets download grassknoted/asl-alphabet

 98% 1.01G/1.03G [00:09<00:00, 109MB/s] 
100% 1.03G/1.03G [00:09<00:00, 117MB/s]


In [ ]:
%%capture
! unzip asl-alphabet.zip

In [ ]:
#prep

import numpy as np
import seaborn as sns
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os
import tensorflow as tf

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  './asl_alphabet_train/asl_alphabet_train/',
  validation_split=0.2,
  subset="training",
  color_mode = 'grayscale',
  seed=123,
  image_size=(200, 200),
  batch_size=100)

Found 87000 files belonging to 29 classes.
Using 69600 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  './asl_alphabet_train/asl_alphabet_train/',
  validation_split=0.2,
  subset="validation",
  color_mode = 'grayscale',
  seed=123,
  image_size=(200, 200),
  batch_size=100)

Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [ ]:
num_classes = len(class_names)
#defining the model

from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

# number of possible label values
# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(200, 200, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(num_classes, activation='softmax'))

opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


696/696 [==============================] - 66s 92ms/step - loss: 0.0044 - accuracy: 0.9988 - val_loss: 0.0464 - val_accuracy: 0.9859
Epoch 2/5
696/696 [==============================] - 64s 92ms/step - loss: 0.0034 - accuracy: 0.9989 - val_loss: 0.1117 - val_accuracy: 0.9779
Epoch 3/5
696/696 [==============================] - 65s 93ms/step - loss: 0.0039 - accuracy: 0.9989 - val_loss: 0.0135 - val_accuracy: 0.9954
Epoch 4/5
696/696 [==============================] - 65s 93ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.1721 - val_accuracy: 0.9634
Epoch 5/5
696/696 [==============================] - 65s 93ms/step - loss: 0.0037 - accuracy: 0.9991 - val_loss: 0.1783 - val_accuracy: 0.9675


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save('drive/MyDrive/loweresModelTest')

INFO:tensorflow:Assets written to: drive/MyDrive/loweresModelTest/assets


In [ ]:
#Show predicition for one image

import tensorflow as tf
import cv2
import numpy as np


CATEGORIES = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "del", "nothing", "space"]


def prepare(filepath):
    IMG_SIZE = 200
    img_array = cv2.imread(filepath, 0)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1) 


#model = tf.keras.models.load_model("drive/MyDrive/loweresModelTest")

prediction = model.predict([prepare("./O_personaltest.jpg")])

print(CATEGORIES[int(np.argmax(prediction[0]))])

Z


In [ ]:
prediction = model.predict([prepare("./A_personaltest.jpg")])

print(CATEGORIES[int(np.argmax(prediction[0]))])

C
